# PFRA 

### _Development Notebook_
--- 

Tests using scrapbook

In [1]:
import sys;sys.path.append('../')

In [22]:
from hecrasio.core import *
from hecrasio.qaqc import *
import scrapbook as sb
import pandas as pd

#### Read model output data 

*from AWS*

In [6]:
nb = "SB-QAQC-FluvialTest.ipynb"

In [20]:
scraps = list(sb.read_notebook(nb).scraps)

### 1A. Plan Information

In [23]:
pd.read_json(scraps[0])

,Results
Base Output Interval,15MIN
Computation Time Courant Method,Representative Length/Velocity
Computation Time Step Base,30SEC
Computation Time Step Count To Double,2
Computation Time Step Max Courant,2
Computation Time Step Max Doubling,2
Computation Time Step Max Halving,1
Computation Time Step Min Courant,0.5
Flow Filename,DC_F01_NBR.u01
Flow Title,E0004


### 1B. Plan Parameters

In [24]:
pd.read_json(scraps[1])

,Results
1D Cores,0
1D Methodology,Finite Difference
1D2D Flow Tolerance,0.1
1D2D MaxIter,0
1D2D Minimum Flow Tolerance,1
1D2D WS Tolerance,0.01
2D Boundary Condition Ramp Up Fraction,[0.1000000015]
2D Boundary Condition Volume Check,[False]
2D Cores,0
2D Coriolis,False


### 2A. Computational Summary

In [25]:
pd.read_json(scraps[2])

,Results
Computation Time DSS,00:00:00
Computation Time Total,03:11:15
Maximum WSEL Error,0
Run Time Window,15JUL2019 03:58:00 to 15JUL2019 07:09:15
Solution,Unsteady Finished Successfully
Time Solution Went Unstable,00:00:00
Time Stamp Solution Went Unstable,Not Applicable
Vol Accounting,Volume Accounting in Acre Feet
Vol Accounting 2D Bdry,0
Vol Accounting Cell End,99553.8


# END